# Best config params were chosen, now running it 5 times with different subsets of train set as validation set

In [1]:
from fastai.text import * 

In [2]:
from ulmfit_experiments import experiments
from ulmfit_experiments import sentencepiece_tok as spm
from ulmfit_experiments.runner import MongoExperimentRunner
from sklearn.metrics import *

In [3]:
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

In [4]:
runner = MongoExperimentRunner(experiments.ExperimentCls.factory, '/home/tomasz/nlp/trained_models/')

In [5]:
db = runner._get_db()

In [6]:
source_id = 137

In [7]:
source_config = db[runner.completed_collection].find_one(source_id)

In [8]:
c = {'type': deepcopy(source_config['type']), 'params': deepcopy(source_config['params'])}

In [9]:
c

{'type': 'Poleval1',
 'params': {'dataset_path': '/home/tomasz/nlp/poleval/csv/',
  'encoder_subdir': 'encoder_tweets_spm_basic',
  'training_phases': [{'freeze_to': -1, 'cyc_len': 1, 'lr_max_last': 0.01},
   {'freeze_to': -2, 'cyc_len': 1, 'lr_max_last': 0.01},
   {'freeze_to': -3, 'cyc_len': 1, 'lr_max_last': 0.005},
   {'freeze_to': -6, 'cyc_len': 10, 'lr_max_last': 0.001}],
  'aggregation_class': 'BranchingAttentionAggregation',
  'aggregation_params': {'att_hid_layers': [50],
   'att_dropouts': 0.2,
   'agg_dim': 50},
  'cv_fold_num': 0,
  'drop_mult': 0.5,
  'rnn_output_layers': [-1],
  'bidir': False,
  'max_len': 1400,
  'bs': 80,
  'metrics': ['accuracy', 'f1'],
  'callbacks': ['average_precision_score'],
  'calc_test_score': False}}

In [10]:
# ex = experiments.ExperimentCls.factory(c['type'], c['params'])

In [40]:
# results, learn = ex.run()

In [46]:
for fold in range(1, 5): 
    print('fold ' +  str(fold))
    c = {'type': deepcopy(source_config['type']), 'params': deepcopy(source_config['params'])}
    c['params']['cv_fold_num'] = fold
    ex = experiments.ExperimentCls.factory(c['type'], c['params'])
    results, learn = ex.run()
    learn.save('poleval-submission-'+str(fold))
    
    preds = learn.get_preds(DatasetType.Valid, ordered=True)
    dfs = ex.get_dfs(fold)
    target = dfs[1][ex.label_col]
    predicted_proba = preds[0][:, 1].numpy()
    fscores = pd.Series({t: f1_score(target, predicted_proba > t) for t in np.arange(0.01,0.99, 0.01)})
    fscores.to_csv(f'/home/tomasz/nlp/poleval/submission/fold{fold}-valid-fscores.csv')
    
    preds = learn.get_preds(DatasetType.Test, ordered=True)
    predicted_proba = preds[0][:, 1].numpy()
    pd.Series(predicted_proba).to_csv(f'/home/tomasz/nlp/poleval/submission/fold{fold}-probas.csv')


0.5211726384364821
